# 📊 QML Strategy Forensic Dashboard

**The 100-Page Report**: Complete validation analysis for the QML Strategy.

---

## Setup

This notebook automatically loads results from `results/autopsy_qml_v1/`.

In [ ]:
# Configuration
RESULTS_DIR = "results/autopsy_qml_v1"
LOAD_LATEST = True  # Set to False and specify EXPERIMENT_ID to load specific experiment
EXPERIMENT_ID = None  # Set this if LOAD_LATEST = False

In [ ]:
# Imports
import sys
sys.path.insert(0, '.')

import json
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML, Markdown, IFrame

# QML Validation Modules
from src.validation import VRDDatabase
from src.reporting import ReportVisualizer, DossierGenerator
from src.deployment import DeploymentGatekeeper

# Enable matplotlib inline
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('dark_background')

print("✅ All modules loaded successfully")

---
## 1. Load Experiment Results

In [ ]:
# Initialize database connection
db_path = Path(RESULTS_DIR) / "vrd.db"

if db_path.exists():
    db = VRDDatabase(str(db_path))
    
    # Load experiment
    if LOAD_LATEST:
        experiment = db.get_latest_experiment()
        if experiment:
            EXPERIMENT_ID = experiment.experiment_id
    else:
        experiment = db.get_experiment(EXPERIMENT_ID) if EXPERIMENT_ID else None
    
    if experiment:
        print(f"📋 Loaded Experiment: {experiment.strategy_name}")
        print(f"   ID: {experiment.experiment_id}")
        print(f"   Date: {experiment.timestamp}")
        print(f"   Status: {experiment.status}")
    else:
        print("❌ No experiment found. Run: python run_autopsy.py")
else:
    experiment = None
    print(f"❌ Database not found at {db_path}")
    print("Run: python run_autopsy.py")

---
## 2. Executive Summary

In [ ]:
if experiment:
    # Display key metrics
    sharpe = experiment.aggregate_sharpe
    max_dd = experiment.aggregate_max_dd
    trades = experiment.total_oos_trades
    p_value = experiment.sharpe_p_value or 1.0
    var_95 = experiment.var_95 or 0
    kill_prob = experiment.kill_switch_prob or 0
    
    # Determine verdict
    if p_value < 0.05 and sharpe > 1.0:
        verdict = "✅ DEPLOY"
        verdict_color = "#27ae60"
    elif p_value < 0.10 and sharpe > 0.5:
        verdict = "⚠️ CAUTION"
        verdict_color = "#f39c12"
    else:
        verdict = "❌ REJECT"
        verdict_color = "#e74c3c"
    
    summary_html = f"""
    <div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); 
                padding: 30px; border-radius: 15px; color: white; margin: 10px 0;">
        <h2 style="text-align: center; margin-bottom: 20px;">🎯 Executive Summary</h2>
        <div style="display: flex; justify-content: space-around; flex-wrap: wrap;">
            <div style="text-align: center; padding: 15px; min-width: 120px;">
                <div style="font-size: 2em; color: {verdict_color}; font-weight: bold;">{verdict}</div>
                <div style="color: #888; font-size: 0.9em;">Verdict</div>
            </div>
            <div style="text-align: center; padding: 15px; min-width: 120px;">
                <div style="font-size: 2em; color: #00d4ff;">{sharpe:.3f}</div>
                <div style="color: #888; font-size: 0.9em;">Sharpe Ratio</div>
            </div>
            <div style="text-align: center; padding: 15px; min-width: 120px;">
                <div style="font-size: 2em; color: #e74c3c;">{max_dd:.1f}%</div>
                <div style="color: #888; font-size: 0.9em;">Max Drawdown</div>
            </div>
            <div style="text-align: center; padding: 15px; min-width: 120px;">
                <div style="font-size: 2em; color: #3498db;">{p_value:.4f}</div>
                <div style="color: #888; font-size: 0.9em;">p-value</div>
            </div>
            <div style="text-align: center; padding: 15px; min-width: 120px;">
                <div style="font-size: 2em; color: #9b59b6;">{trades}</div>
                <div style="color: #888; font-size: 0.9em;">Trades</div>
            </div>
        </div>
        <hr style="border-color: rgba(255,255,255,0.1); margin: 20px 0;">
        <div style="display: flex; justify-content: center; gap: 40px;">
            <div style="text-align: center;">
                <span style="color: #f1c40f;">VaR 95%:</span> {var_95:.1f}%
            </div>
            <div style="text-align: center;">
                <span style="color: #e74c3c;">Ruin Prob:</span> {kill_prob:.1%}
            </div>
        </div>
    </div>
    """
    display(HTML(summary_html))

---
## 3. View HTML Dossier

In [ ]:
# Find and display the dossier
if experiment:
    dossier_files = list(Path(RESULTS_DIR).glob("dossier_*.html"))
    
    if dossier_files:
        dossier_path = dossier_files[0]
        print(f"📄 Dossier: {dossier_path}")
        display(IFrame(src=str(dossier_path), width="100%", height=800))
    else:
        print("No dossier file found. Generate with DossierGenerator.")

---
## 4. Monte Carlo Simulation

In [ ]:
visualizer = ReportVisualizer()

# Simulate equity paths for demonstration
if experiment:
    n_sims = 1000
    n_trades = max(50, experiment.total_oos_trades)
    
    # Use experiment metrics to generate realistic paths
    mean_ret = experiment.aggregate_return / n_trades if n_trades > 0 else 0.5
    std_ret = abs(mean_ret) * 2  # Approximate
    
    returns = np.random.randn(n_sims, n_trades) * std_ret + mean_ret / 100
    equity_paths = 100000 * np.cumprod(1 + returns, axis=1)
    
    fig = visualizer.plot_monte_carlo_cones(
        equity_paths,
        initial_capital=100000,
        title=f"Monte Carlo: {experiment.strategy_name}"
    )
    plt.show()

---
## 5. Drawdown Analysis

In [ ]:
if experiment and experiment.total_oos_trades > 0:
    # Generate representative equity curve
    n_points = max(100, experiment.total_oos_trades * 5)
    mean_ret = experiment.aggregate_return / experiment.total_oos_trades / 100
    returns = np.random.randn(n_points) * 0.01 + mean_ret
    equity_curve = 100000 * np.cumprod(1 + returns)
    
    fig = visualizer.plot_drawdown_chart(
        equity_curve,
        title=f"Drawdown Analysis: {experiment.strategy_name}"
    )
    plt.show()

---
## 6. Deployment Readiness Check

In [ ]:
gatekeeper = DeploymentGatekeeper()

if experiment:
    # Create result object from experiment
    class MockResult:
        pass
    
    mock = MockResult()
    mock.oos_sharpe = experiment.aggregate_sharpe
    mock.oos_max_dd = experiment.aggregate_max_dd
    mock.total_trades = experiment.total_oos_trades
    mock.sharpe_p_value = experiment.sharpe_p_value or 1.0
    mock.monte_carlo_result = None
    mock.walk_forward_result = None
    mock.regime_performance = experiment.regime_metrics or {}
    
    readiness = gatekeeper.check_readiness(mock)
    
    status_color = "#27ae60" if readiness.is_ready else "#e74c3c"
    status_text = "✅ READY FOR DEPLOYMENT" if readiness.is_ready else "❌ NOT READY"
    
    readiness_html = f"""
    <div style="background: {status_color}; padding: 20px; border-radius: 10px; 
                color: white; text-align: center; margin: 20px 0;">
        <h2 style="margin: 0;">{status_text}</h2>
        <p style="margin: 10px 0 0 0;">Score: {readiness.overall_score:.0f}/100</p>
    </div>
    """
    display(HTML(readiness_html))
    
    # Details
    details_html = f"""
    <div style="display: flex; gap: 20px; margin-top: 15px;">
        <div style="flex: 1; background: rgba(39,174,96,0.2); padding: 15px; border-radius: 10px;">
            <h4 style="color: #27ae60;">✅ Passed ({len(readiness.passed_checks)})</h4>
            <ul style="margin: 0; padding-left: 20px;">{''.join(f'<li>{c}</li>' for c in readiness.passed_checks)}</ul>
        </div>
        <div style="flex: 1; background: rgba(231,76,60,0.2); padding: 15px; border-radius: 10px;">
            <h4 style="color: #e74c3c;">❌ Failed ({len(readiness.failed_checks)})</h4>
            <ul style="margin: 0; padding-left: 20px;">{''.join(f'<li>{c}</li>' for c in readiness.failed_checks)}</ul>
        </div>
    </div>
    """
    display(HTML(details_html))

---
## 7. Fold-by-Fold Analysis

In [ ]:
if experiment and experiment.fold_results:
    folds_df = pd.DataFrame(experiment.fold_results)
    
    if not folds_df.empty:
        print("📊 Walk-Forward Fold Results:")
        display(folds_df.style.format(precision=3))
else:
    print("No fold data available.")

---
## 8. All Experiments Comparison

In [ ]:
if 'db' in dir() and db:
    recent = db.get_recent_experiments(limit=10)
    
    if recent:
        comparison_data = []
        for exp in recent:
            comparison_data.append({
                'ID': exp.experiment_id[:8] + '...',
                'Strategy': exp.strategy_name,
                'Sharpe': exp.aggregate_sharpe,
                'Max DD': f"{exp.aggregate_max_dd:.1f}%",
                'Trades': exp.total_oos_trades,
                'p-value': f"{(exp.sharpe_p_value or 1.0):.4f}",
                'Status': exp.status,
            })
        
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df.style.format(precision=3))
    else:
        print("No experiments found.")

---

## 📝 Next Steps

1. **If DEPLOY**: Start paper trading with 25% position size
2. **If CAUTION**: Run extended validation or adjust parameters
3. **If REJECT**: Review statistical findings and iterate on strategy

---
*Generated by QML Validation Framework*